In [2]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from dtreeviz.trees import *
import sklearn

def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

traindf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
traindf['Train'] = True
testdf['Train'] = False
togeth = pd.concat([traindf,testdf])


/home/jakeunix/miniforge3/envs/sptitanic/lib/python3.11/site-packages/fastbook/__init__.py:11: UserWarning: Missing `ipywidgets` - please install it
  except ModuleNotFoundError: warn("Missing `ipywidgets` - please install it")
/home/jakeunix/miniforge3/envs/sptitanic/lib/python3.11/site-packages/fastbook/__init__.py:13: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")


In [3]:
def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    return df
togeth = split_group_number(togeth)

def split_names(df):
    # Create new columns for first name and last name
    df['FirstName'] = df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else pd.NA)
    df['LastName'] = df['Name'].apply(lambda x: x.split()[1] if pd.notna(x) else pd.NA)
    df.drop(columns = ['Name'],inplace = True)
    return df
togeth = split_names(togeth)


def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df
togeth = split_cabin(togeth)


In [4]:
spending = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
togeth['Spending'] = togeth[spending].sum(axis = 1)
missing = togeth.copy()
togeth.dropna(inplace = True)

In [4]:
def values(list,check,df):
    for item in list:
        if item[1] == 'same':
            df = df[df[item[0]] == item[2]]
        elif item[1] == 'less':
            df = df[df[item[0]] < item[2]]
        else :
            df = df[df[item[0]] > item[2]]
    return df[check].value_counts()


values([['HomePlanet','same','Earth'],['CabinDeck','same', 'F']],'CryoSleep',togeth)

CryoSleep
False    1272
True        8
Name: count, dtype: int64

In [13]:
togeth[togeth.HomePlanet == 'Mars'].CryoSleep.value_counts()

CryoSleep
False    849
True     518
Name: count, dtype: int64

In [14]:
togeth[togeth.Spending == 0].CryoSleep.value_counts()

CryoSleep
True     2332
False     404
Name: count, dtype: int64

In [15]:
togeth[togeth.CabinDeck == 'G'].CryoSleep.value_counts()

CryoSleep
True     1072
False     901
Name: count, dtype: int64

In [5]:
1272/1280

0.99375

In [302]:
def proportion_same_homeplanet_with_lastname(df):
    same_homeplanet_count = 0
    total_count = 0
    for group, group_df in df.groupby(['LastName','Cabin','CryoSleep']):
        if group_df['Destination'].nunique() == 1:
            same_homeplanet_count += len(group_df)
        total_count += len(group_df)

    proportion = same_homeplanet_count / total_count if total_count > 0 else 0
    return proportion

# Call the function to get the proportion
proportion = proportion_same_homeplanet_with_lastname(togeth)
print(f"The proportion of people with the same home planet as everyone with the same last name is: {proportion:.2%}")


The proportion of people with the same home planet as everyone with the same last name is: 90.96%


In [303]:
def proportion_same_destination_in_cabin(df):
    same_destination_count = 0
    total_count = 0
    for cabin, cabin_df in df.groupby('Cabin'):
        if len(cabin_df) >= 2:  # Check if the cabin has at least 2 people
            cabin_without_self = cabin_df[cabin_df['PassengerId'] != cabin_df['PassengerId'].values[0]]
            unique_destinations = cabin_without_self['Destination'].nunique()
            if unique_destinations == 1:
                if cabin_without_self['Destination'].values[0] == cabin_df['Destination'].values[0]:
                    same_destination_count += 1
                total_count += 1
    proportion = same_destination_count / total_count if total_count > 0 else 0
    return proportion

# Call the function to get the proportion
proportion = proportion_same_destination_in_cabin(togeth)
print(f"The proportion of people with the same destination in their cabin (cabin size >= 2, excluding self) is: {proportion:.2%}")


The proportion of people with the same destination in their cabin (cabin size >= 2, excluding self) is: 56.00%


In [307]:
missing.isna().sum()

PassengerId        0
HomePlanet       288
CryoSleep        310
Cabin            299
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Transported     4277
Train              0
Group              0
GroupNumber        0
FirstName        294
LastName         294
CabinDeck        299
CabinSide         39
CabinNum         299
Spending           0
dtype: int64

In [306]:
def fillna_cabinside(df):
    groups = df.groupby('Group')
    for name, group in groups:
        group_side = group['CabinSide'].dropna().unique()
        if len(group_side) == 1:
            side_value = group_side[0]
            df.loc[group.index, 'CabinSide'] = side_value
    return df

# Call the function to fill NaN values in 'CabinSide'
missing = fillna_cabinside(missing)


In [314]:
def likelihood_same_cabinside_in_group(df):
    valid_groups = df.groupby('LastName').filter(lambda x: len(x) > 1)
    valid_group_count = 0
    same_cabinside_count = 0

    for group, group_df in valid_groups.groupby('LastName'):
        group_size = len(group_df)
        same_cabinside = group_df['CabinDeck'].nunique() == 1

        if same_cabinside:
            same_cabinside_count += group_size
        valid_group_count += group_size

    likelihood = same_cabinside_count / valid_group_count if valid_group_count > 0 else 0
    return likelihood

# Call the function to get the likelihood
likelihood = likelihood_same_cabinside_in_group(togeth)
print(f"The likelihood that, given someone not alone in their group, everyone shares the same cabin side is: {likelihood:.2%}")


The likelihood that, given someone not alone in their group, everyone shares the same cabin side is: 19.55%


In [6]:
traindf

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Train
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,True
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,True
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,True
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,True
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,True


In [7]:
def proportion_same_cabinside_in_group(df):
    same_cabinside_count = 0
    total_count = 0
    grouped = df.groupby('Group')
    
    for group, group_df in grouped:
        unique_cabinside_values = group_df['CabinSide'].dropna().unique()
        
        if len(unique_cabinside_values) == 1:
            same_cabinside_count += len(group_df)
        
        total_count += len(group_df)
    
    proportion = same_cabinside_count / total_count
    return proportion

proportion = proportion_same_cabinside_in_group(togeth)
print(f"The proportion of people with the same 'CabinSide' in the same group is: {proportion:.2%}")


The proportion of people with the same 'CabinSide' in the same group is: 100.00%
